# Global Imports

In [ ]:
import os
import pprint
import sys
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import matplotlib.pyplot as plt
import numpy as np
from matplotlib import gridspec

from utils_002 import gen_xyz, load_cfg


def read_simlog(simlog_file: Path) -> Dict[str, np.ndarray]:
    """
    Reads a simulation log file and returns its data as a dictionary
    of NumPy arrays.

    Args:
        simlog_file: The path to the 'SimLog.txt' file.

    Returns:
        A dictionary containing simulation data.
    """
    try:
        lines = simlog_file.read_text(encoding="utf-8").splitlines()
    except FileNotFoundError:
        print(f"Error: The file {simlog_file} was not found.")
        return {}

    simlog_data = []
    for line in lines:
        stripped_line = line.strip()
        try:
            data_points = [float(val) for val in stripped_line.split(":")]
            simlog_data.append(np.array(data_points))
        except (ValueError, IndexError) as e:
            # print(
            #     f"Warning: Skipping malformed line in {simlog_file}: '{line.strip()}' - {e}"
            # )
            simlog_data.append(np.array([]))

    keys = [
        "n_gas",
        "n_crystal",
        "concentration",
        "delta_gibbs",
        "energy_change",
        "crystal_sx",
        "crystal_sy",
        "crystal_sz",
        "tri",
    ]

    if len(simlog_data) != len(keys):
        print(
            f"Warning: Mismatch between expected data lines ({len(keys)}) and actual lines ({len(simlog_data)}) in {simlog_file}."
        )
        return {}

    return dict(zip(keys, simlog_data))


def process_directory(dir_path: Path) -> Dict[str, Dict[str, np.ndarray]]:
    """
    Processes a directory containing simulation results,
    finding and parsing 'InitSettings.ini' and 'SimLog.txt' files.

    Args:
        dir_path: The root directory path to process.

    Returns:
        A dictionary where keys are sub-directory IDs and values are
        the parsed simulation data.
    """
    cfg_file = dir_path / "InitSettings.ini"
    main_simlog_file = dir_path / "SimLog.txt"
    if not cfg_file.exists():
        raise FileNotFoundError(f"Missing InitSettings.ini in {dir_path}")
    if not main_simlog_file.exists():
        raise FileNotFoundError(f"Missing SimLog.txt in {dir_path}")

    cfg = load_cfg(
        path_to_config=cfg_file,
        com_line="/////////////////////////////// | GENERAL INFO | ///////////////////////////////",
    )

    # sx, sy, sz = cfg["Sx"], cfg["Sy"], cfg["Sz"]
    # x, y, z = gen_xyz(sx, sy, sz, mode=2)

    try:
        main_id = dir_path.name.split("_", 1)[0]
    except IndexError:
        print(f"Warning: Could not extract ID from {dir_path.name}")
        main_id = dir_path.name

    items_data = {
        "cfg": cfg,
        "main": {"ID": main_id, "data": read_simlog(main_simlog_file)},
        "sub": {},
    }
    for sub_dir_path in dir_path.glob("**/"):
        states_file = sub_dir_path / "TimeStates.txt"
        simlog_file = sub_dir_path / "SimLog.txt"
        if states_file.exists() and simlog_file.exists():
            print(f"--> Processing: {sub_dir_path}")

            try:
                sub_id = sub_dir_path.name.split("_", 1)[0]
            except IndexError:
                print(
                    f"Warning: Could not extract IDs from {dir_path.name} or {sub_dir_path.name}"
                )
                sub_id = sub_dir_path.name

            items_data["sub"][sub_id] = {"ID": sub_id, "data": read_simlog(simlog_file)}

    return items_data

In [ ]:
BASE_PATH = Path(  #
    r"1754236522824104_Mode2_N100_X50Y200Z50_T3e2_C9.58767e-8_Nt1e11_Pb0.05"
    #
)

try:
    print(f"Processing base directory: {BASE_PATH}")
    all_data = process_directory(BASE_PATH)

except FileNotFoundError as e:
    print(f"Error: {e}")
    sys.exit(1)
except Exception as e:
    print(f"An unexpected error occurred: {e}")
    sys.exit(1)

In [ ]:
# pprint.pprint(all_data)

In [ ]:
# MAX_SUB_LEN = max(
#     [item["data"]["crystal_sy"].size for name, item in all_data["sub"].items()]
# )

# tri_global = None

# for name, item in all_data["sub"].items():
#     ID, DATA = item["ID"], item["data"]

#     tri = item["data"]["tri"]

#     if item["data"]["tri"].size == MAX_SUB_LEN:
#         tri_global = tri.copy()

# base = np.zeros(MAX_SUB_LEN)

# new_sub = {}

# for name, item in all_data["sub"].items():
#     ID, DATA = item["ID"], item["data"]

#     for k, v in DATA.items():
#         base.fill(0.0)
#         if k == "tri":
#             base = tri_global.copy()
#         base[: v[:-1].size] = v[:-1]
#         DATA[k] = base.copy()

#     new_sub[name] = {"ID": ID, "data": DATA}

# all_data["sub"] = new_sub
# del new_sub, base

In [ ]:
# pprint.pprint(all_data)

In [ ]:
fig, ax = plt.subplots(1, figsize=(8, 4), dpi=150)

for name, item in all_data["sub"].items():
    ID, DATA = item["ID"], item["data"]
    ax.plot(tri_global, DATA["crystal_sy"])

ax.grid(True)
plt.show()
plt.close()

In [ ]:
plt.rcParams.update(
    {
        "font.family": "serif",
        "font.serif": ["Times New Roman"],
        "font.size": 8,
        "axes.labelsize": 8,
        "axes.titlesize": 8,
        "legend.fontsize": 7,
        "xtick.labelsize": 7,
        "ytick.labelsize": 7,
        "lines.linewidth": 0.5,
        "lines.markersize": 2.5,
        "xtick.direction": "in",
        "ytick.direction": "in",
        "axes.grid": True,
        "grid.linestyle": ":",
        "grid.linewidth": 0.4,
    }
)

In [ ]:
path_to_dir = Path(  #
    r"1754236522824104_Mode2_N100_X50Y200Z50_T3e2_C9.58767e-8_Nt1e11_Pb0.05\Graphs"
    #
)

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(tri_global)
Y = np.mean(
    np.vstack(
        [*map(lambda item: item["data"]["energy_change"], all_data["sub"].values())]
    ),
    axis=0,
)
datasets = [
    ("Data", X[:-1], Y[:-1], "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Average change\nin Total Surface Energy\nacross the ensemble")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_01.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(all_data["main"]["data"]["tri"])
Y = np.copy(all_data["main"]["data"]["concentration"])
datasets = [
    ("Data", X[:-1], Y[:-1], "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Concentration ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_02.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(all_data["main"]["data"]["tri"])
Y = np.copy(all_data["cfg"]["N_tot"] - all_data["main"]["data"]["n_gas"])
datasets = [
    ("Data", X[:-1], Y[:-1], "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Ntotal - Ngas ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_03.png", format="png")
plt.show()
plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(tri_global)
Y = np.sum(
    np.vstack(
        [*map(lambda item: item["data"]["n_crystal"] > 0.0, all_data["sub"].values())]
    ),
    axis=0,
)
datasets = [
    ("Data", X[:-1], Y[:-1], "solid", 0.60, "o", ".00"),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        # label=label,
    )


ax1.set_xlabel("Number of $TRI$")
ax1.set_ylabel("Value of\nEnsemble [ Alive Particles ]")
# ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_04.png", format="png")
plt.show()
plt.close()

In [ ]:
# fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

# X = np.copy(tri_global)
# Yx = np.mean(
#     np.vstack(
#         [*map(lambda item: item["data"]["crystal_sx"], all_data["sub"].values())]
#     ),
#     axis=0,
# )
# Yy = np.mean(
#     np.vstack(
#         [*map(lambda item: item["data"]["crystal_sy"], all_data["sub"].values())]
#     ),
#     axis=0,
# )
# Yz = np.mean(
#     np.vstack(
#         [*map(lambda item: item["data"]["crystal_sz"], all_data["sub"].values())]
#     ),
#     axis=0,
# )
# Yx_std = np.std(Yx, ddof=1)
# Yy_std = np.std(Yy, ddof=1)
# Yz_std = np.std(Yz, ddof=1)
# datasets = [
#     (f"X axis ( STD: {Yx_std:.2f} )", X[:-1], Yx[:-1], "solid", 0.60, "s", ".50"),
#     (f"Y axis ( STD: {Yy_std:.2f} )", X[:-1], Yy[:-1], "solid", 0.60, "o", ".25"),
#     (f"Z axis ( STD: {Yz_std:.2f} )", X[:-1], Yz[:-1], "solid", 0.60, "^", ".75"),
# ]
# for label, x, y, ls, lw, marker, color in datasets:
#     ax1.plot(
#         x,
#         y,
#         linewidth=lw,
#         linestyle=ls,
#         color=color,
#         marker=marker,
#         markerfacecolor="white",
#         markeredgecolor=color,
#         markeredgewidth=0.3,
#         markevery=X.size // 30,
#         label=label,
#     )


# ax1.set_xlabel("Number of $TRI$")
# ax1.set_ylabel("Average axial particle size\nin the Ensemble")
# ax1.legend(loc="best", frameon=True)

# ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
# ax1.tick_params(length=2, width=0.5)

# plt.tight_layout()

# plt.savefig(path_to_dir / "G_05.png", format="png")
# plt.show()
# plt.close()

In [ ]:
mask_alive = np.vstack(
    [*map(lambda item: item["data"]["n_crystal"] > 0.0, all_data["sub"].values())]
)

alive = mask_alive.sum(axis=0)[:-1]

Sx = np.vstack(
    [*map(lambda item: item["data"]["crystal_sx"], all_data["sub"].values())]
)[:, :-1]
Sy = np.vstack(
    [*map(lambda item: item["data"]["crystal_sy"], all_data["sub"].values())]
)[:, :-1]
Sz = np.vstack(
    [*map(lambda item: item["data"]["crystal_sz"], all_data["sub"].values())]
)[:, :-1]

Sx_mean = Sx.sum(axis=0) / living
Sy_mean = Sy.sum(axis=0) / living
Sz_mean = Sz.sum(axis=0) / living

Sx2_mean = (Sx ** 2.0).sum(axis=0) / living
Sy2_mean = (Sy ** 2.0).sum(axis=0) / living
Sz2_mean = (Sz ** 2.0).sum(axis=0) / living

Sx_mean3 = Sx_mean ** 3.0
Sy_mean3 = Sy_mean ** 3.0
Sz_mean3 = Sz_mean ** 3.0

Sx_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sx_mean, mask_alive)])
Sy_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sy_mean, mask_alive)])
Sz_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sz_mean, mask_alive)])

# Sx2_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sx2_mean, mask_alive)])
# Sy2_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sy2_mean, mask_alive)])
# Sz2_mean_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sz2_mean, mask_alive)])

# Sx_mean3_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sx_mean3, mask_alive)])
# Sy_mean3_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sy_mean3, mask_alive)])
# Sz_mean3_STD = np.array([np.std(i[j], ddof=1) for i, j in zip(Sz_mean3, mask_alive)])

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(tri_global)
Yx = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sx"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
)
Yy = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sy"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
)
Yz = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sz"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
)
# Yx_std = np.std(Yx, ddof=1)
# Yy_std = np.std(Yy, ddof=1)
# Yz_std = np.std(Yz, ddof=1)
datasets = [
    (
        # f"X axis ( STD: {Yx_std:.2f} )",
        f"X axis",
        X[: living.size],
        Yx,
        "solid",
        0.60,
        "s",
        ".50",
    ),
    (
        # f"Y axis ( STD: {Yy_std:.2f} )",
        f"Y axis",
        X[: living.size],
        Yy,
        "solid",
        0.60,
        "o",
        ".25",
    ),
    (
        # f"Z axis ( STD: {Yz_std:.2f} )",
        f"Z axis",
        X[: living.size],
        Yz,
        "solid",
        0.60,
        "^",
        ".75",
    ),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
# ax1.set_ylabel("Average axial particle size\nin the Ensemble\n( Alive only )") # "G_06.png"
# ax1.set_ylabel("Average axial particle size\nin the Ensemble")  # "G_06_1.png"
ax1.set_ylabel("Average axial particle size\nin the Ensemble")  # "G_06_2.png" NO STD
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_06_2.png", format="png")
plt.show()
plt.close()

In [ ]:
# fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

# X = np.copy(tri_global)
# Yx = (
#     np.mean(
#         np.vstack(
#             [*map(lambda item: item["data"]["crystal_sx"], all_data["sub"].values())]
#         ),
#         axis=0,
#     )
#     ** 3
# )
# Yy = (
#     np.mean(
#         np.vstack(
#             [*map(lambda item: item["data"]["crystal_sy"], all_data["sub"].values())]
#         ),
#         axis=0,
#     )
#     ** 3
# )
# Yz = (
#     np.mean(
#         np.vstack(
#             [*map(lambda item: item["data"]["crystal_sz"], all_data["sub"].values())]
#         ),
#         axis=0,
#     )
#     ** 3
# )
# Yx_std = np.std(Yx, ddof=1)
# Yy_std = np.std(Yy, ddof=1)
# Yz_std = np.std(Yz, ddof=1)
# datasets = [
#     (f"X axis ( STD: {Yx_std:.2f} )", X[:-1], Yx[:-1], "solid", 0.60, "s", ".50"),
#     (f"Y axis ( STD: {Yy_std:.2f} )", X[:-1], Yy[:-1], "solid", 0.60, "o", ".25"),
#     (f"Z axis ( STD: {Yz_std:.2f} )", X[:-1], Yz[:-1], "solid", 0.60, "^", ".75"),
# ]
# for label, x, y, ls, lw, marker, color in datasets:
#     ax1.plot(
#         x,
#         y,
#         linewidth=lw,
#         linestyle=ls,
#         color=color,
#         marker=marker,
#         markerfacecolor="white",
#         markeredgecolor=color,
#         markeredgewidth=0.3,
#         markevery=X.size // 30,
#         label=label,
#     )


# ax1.set_xlabel("Number of $TRI$")
# ax1.set_ylabel("[ Average axial particle size ]^3\nin the Ensemble")
# ax1.legend(loc="best", frameon=True)

# ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
# ax1.tick_params(length=2, width=0.5)

# plt.tight_layout()

# plt.savefig(path_to_dir / "G_07.png", format="png")
# plt.show()
# plt.close()

In [ ]:
fig, ax1 = plt.subplots(1, 1, figsize=(3.5, 2), dpi=600, sharex=True)

X = np.copy(tri_global)
living = np.sum(
    np.vstack(
        [*map(lambda item: item["data"]["n_crystal"] > 0.0, all_data["sub"].values())]
    ),
    axis=0,
)[:-1]
Yx = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sx"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
) ** 3
Yy = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sy"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
) ** 3
Yz = (
    np.sum(
        np.vstack(
            [*map(lambda item: item["data"]["crystal_sz"], all_data["sub"].values())]
        ),
        axis=0,
    )[:-1]
    / living
) ** 3
Yx_std = np.std(Yx, ddof=1)
Yy_std = np.std(Yy, ddof=1)
Yz_std = np.std(Yz, ddof=1)
datasets = [
    (
        f"X axis ( STD: {Yx_std:.2f} )",
        X[: living.size],
        Yx,
        "solid",
        0.60,
        "s",
        ".50",
    ),
    (
        f"Y axis ( STD: {Yy_std:.2f} )",
        X[: living.size],
        Yy,
        "solid",
        0.60,
        "o",
        ".25",
    ),
    (
        f"Z axis ( STD: {Yz_std:.2f} )",
        X[: living.size],
        Yz,
        "solid",
        0.60,
        "^",
        ".75",
    ),
]
for label, x, y, ls, lw, marker, color in datasets:
    ax1.plot(
        x,
        y,
        linewidth=lw,
        linestyle=ls,
        color=color,
        marker=marker,
        markerfacecolor="white",
        markeredgecolor=color,
        markeredgewidth=0.3,
        markevery=X.size // 30,
        label=label,
    )


ax1.set_xlabel("Number of $TRI$")
# ax1.set_ylabel("[ Average axial particle size ]^3\nin the Ensemble\n( Alive only )") # "G_08.png"
ax1.set_ylabel("[ Average axial particle size ]^3\nin the Ensemble")  # "G_08_1.png"
ax1.legend(loc="best", frameon=True)

ax1.grid(True, which="both", linestyle=":", linewidth=0.4)
ax1.tick_params(length=2, width=0.5)

plt.tight_layout()

plt.savefig(path_to_dir / "G_08_1.png", format="png")
plt.show()
plt.close()